## Imports and Declerations:

In [2]:
import pickle
import os
import string
import scipy
import pandas as pd
from pandas import HDFStore 
import re
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import clear_output
import numpy as np
from copy import deepcopy
import random
from nltk.stem import PorterStemmer
from collections import Counter
import joblib
InteractiveShell.ast_node_interactivity = "all"


## Helper Functions:

In [3]:

# Remove Punctuation
def remove_punctuation(word):
    return word.translate(word.maketrans('', '', string.punctuation))
printable = set(string.printable)

# Clean Query Term
def clean_word(word):
    # Case Folding
    ps = PorterStemmer()
    word = word.lower()
    # Filter non-ASCII characters
    word = ''.join(filter(lambda x: x in printable, word))
    #     print(word)
    # Remove Punctuations
    if word != '(' and word != ')':
        word = remove_punctuation(word)
#     print(word)
    if re.match('\d+[A-Za-z]+', word):
        word = re.split('\d+', word)[1]
    if re.match('[A-Za-z]+\d+', word):
        word = re.split('\d+', word)[0]


#     print(word)
    word = ps.stem(word)
    #     print(word)
    return word

In [4]:
DOCUMENTS_PATH = ('data', )
STOPWORD_PATH = ('Stopword-List.txt', )

In [98]:
class JSONDocToVec(object):
    def __init__(self, DOCUMENTS_PATH, STOP_WORD_PATH):
        self.ingredients = set()
        self.doc_index = {}
        self.documents_path = DOCUMENTS_PATH
        self.stop_word_path = STOP_WORD_PATH
        self.stop_words = self.load_stop_words()
        self.Xindex = []
        self.vocab_index = self.file_extraction_wrapper(extract_vocab=True)
        vectors = self.file_extraction_wrapper(extract_vectors=True)
        self.files = {}
        
#         self.X = self.vectors[0]
        self.y = vectors[1]
        
        data = pd.DataFrame(vectors[0])
        # # Feature Selection
        # Drop Features with Df < 3
        data.drop([
            col for col, val in data.sum().iteritems() if int(val) <= 3
        ],axis=1,inplace=True)
        data.mul(data.sum().apply(lambda df: np.log10(data.shape[0] / (df + 1))),
         axis=1)
        self.data = scipy.sparse.csr_matrix(data.values)
        # Tf - Idf Calculation
        self.idf = data.sum().apply(lambda df: np.log10(data.shape[0] / (df + 1)))

    def file_extraction_wrapper(self,
                                extract_vocab=False,
                                extract_vectors=False):
        vocab = set()
        docs = {}
        printable = set(string.printable)
        raw_data = []
        if extract_vectors:
            X = []
            y = []
            Xindex = []
        doc_count = 0
        # Printable characters are
        # 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
        # !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c
        ps = PorterStemmer()
        json_files = next(os.walk(os.path.join(self.documents_path)))[2]
        print('dir : ',  list(os.walk(os.path.join(self.documents_path))))
        print(json_files)
        for jfile in json_files:
#             docs_in_c = next(os.walk(os.path.join(self.documents_path, c)))[2]
            if jfile.startswith('test'):
                continue
            print(jfile)
            print(self.documents_path)
            print('filepath : ',(os.path.join(self.documents_path,jfile)))
            
            with open(os.path.join(self.documents_path, jfile),
                          'r') as file1:
                rows = json.load(file1)
                
                for doc in rows:
                    
                    if extract_vectors:
                        doc_vector = np.zeros((len(self.vocab_index)))
#                         doc_name = os.path.join(self.documents_path, c, doc)
                        self.doc_index[doc_count] =doc['id']
                        doc_count+=1
                 
#                         symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
#                         for i in symbols:
#                             line = line.replace(i, ' ')
                    for word in doc['ingredients']:

                        # Case Folding
                        word = word.lower()

                        # Filter non-ASCII characters
                        word = ''.join(
                            filter(lambda x: x in printable, word))

                        if word in self.stop_words:
                            continue

                        # Remove Punctuations
                        word = remove_punctuation(word)

                        if re.match('\d+[A-Za-z]+', word):
                            word = re.split('\d+', word)[1]
                        if re.match('[A-Za-z]+\d+', word):
                            word = re.split('\d+', word)[0]

                        if len(word) == 0 or len(
                                word) == 1 or word == '' or word == ' ':
                            continue
                        if extract_vocab:
                            self.ingredients.add(word)
                        word = ps.stem(word)
#                         print(word)
                        if extract_vocab:
                            
                            vocab.add(word)
                        if extract_vectors:
                            doc_vector[self.vocab_index[word]] += 1

                    if extract_vectors:
                        Xindex.append(doc['id'])
                        X.append(doc_vector)
                        if jfile.startswith('test'):
                            y.append(None)
                        else:
                            y.append(doc['cuisine'])
        if extract_vocab:
            print(f'Vocab Size : {len(vocab)}')
            vocab_list = sorted(list(vocab))
            vocab_hash = dict.fromkeys(vocab_list, 0)
            vocab_index = {
                word: index
                for index, word in enumerate(vocab_list)
            }
            return vocab_index

        if extract_vectors:
            self.Xindex = Xindex
            return (X, y)

    def get_query_vector(self, query_terms):
        ps = PorterStemmer()
        pd_data = pd.DataFrame(self.data.toarray())
        query_vector =pd.Series(pd_data.T[0])
#         print(query_vector.index)
        query_terms = [ps.stem(word.lower()) for word in query_terms]
        for term in query_terms:
            if term in self.vocab_index.keys():
                if self.vocab_index[term] in self.idf.index:
#                     print(term)
                    query_vector.loc[ self.idf.index.get_loc(self.vocab_index[term])] += 1
#         print(query_vector.col[query_vector > 0])
        for index in query_vector.index[query_vector > 0]:
            query_vector.iloc[index] *= self.idf.iloc[index] 
        return query_vector
        
    def load_stop_words(self):
        stop_words = set()
        with open(self.stop_word_path, 'r') as stop_word_file:
            lines = stop_word_file.readlines()
            for line in lines:
                stop_words.add(line.split('\n')[0])
        return stop_words

In [33]:
ingredients = []
with open('./data/train.json') as f:
    rows = json.load(f)
    for recipe in rows:
        ingredients.append(' '.join(recipe['ingredients']))
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(ingredients)
print(X.shape)

(39774, 3051)


In [34]:
def sparse_memory_usage(mat):
    try:
        return mat.data.nbytes + mat.indptr.nbytes + mat.indices.nbytes
    except AttributeError:
        return -1
import scipy
scipy.sparse.save_npz('sparse_matrix.npz', X)

In [46]:

dv = JSONDocToVec(DOCUMENTS_PATH=os.path.join(*DOCUMENTS_PATH),
              STOP_WORD_PATH=os.path.join(*STOPWORD_PATH))


dir :  [('data', [], ['test.json', 'train.json'])]
['test.json', 'train.json']
train.json
data
filepath :  data\train.json
Vocab Size : 6686
dir :  [('data', [], ['test.json', 'train.json'])]
['test.json', 'train.json']
train.json
data
filepath :  data\train.json


In [6]:
X_self = scipy.sparse.csr_matrix(dv.data.values)

def sparse_memory_usage(mat):
    try:
        return mat.data.nbytes + mat.indptr.nbytes + mat.indices.nbytes
    except AttributeError:
        return -1
sparse_memory_usage(X_self)
scipy.sparse.save_npz('sparse_matrix.npz', X_self)




5241004

In [6]:
dv.data = pd.DataFrame(scipy.sparse.load_npz('sparse_matrix.npz').toarray())

NameError: name 'dv' is not defined

In [7]:
loaded_data = pd.DataFrame(scipy.sparse.load_npz('sparse_matrix.npz').toarray())

In [7]:
dv.data

,0,1,2,3,4,5,6,7,8,9,...,3665,3666,3667,3668,3669,3670,3671,3672,3673,3674
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
dv.data.memory_usage(index=True).sum() / (1024 * 1024 * 1024)

1.0890473425388336

In [62]:
sciket_X.memory_usage(index=True).sum() / (1024 * 1024 * 1024)

0.9041315615177155

In [22]:
import sys

def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size
get_size(dv)

9510335

In [49]:
dv.ingredients

{'sweet yellow corn',
 'seedless red grapes',
 'chinese red rice vinegar',
 'vanilla wafers',
 'baby kale',
 'parmigiano',
 'cooked brown rice',
 'fresh onion',
 'vietnamese coriander',
 'fresh lemon',
 'liverwurst',
 'rapeseed oil',
 'idaho potatoes',
 'pancake',
 'chocolate chip cookie dough ice cream',
 'chocolatehazelnut spread',
 'softshelled crabs',
 'chocolate leaves',
 'cooked bone in ham',
 'chocolate extract',
 'rose water',
 'cut up chicken',
 'ground caraway',
 'biscuit mix',
 'spot prawns',
 'tuaca liqueur',
 'morcilla',
 'dried apple',
 'bulb',
 'prepared pie crusts',
 'barilla ovenready lasagne',
 'cold meatloaf',
 'pure maple syrup',
 'fresh mozzarella balls',
 'instant banana cream pudding',
 'crab sticks',
 'diced green chilies',
 'jack',
 'crystal farms reduced fat shredded marble jack cheese',
 'veggies',
 'reduced sodium vegetable stock',
 'skinless chicken breasts',
 'cooki vanilla wafer',
 'vegetarian oyster sauce',
 'halibut fillets',
 'mixed greens',
 'fully co

In [51]:
# Save Vectors
vectors_file_name = 'DV'
pickle.dump(dv, open(vectors_file_name , 'wb'))

dv = pickle.load(dv, open(vectors_file_name , 'rb'))


In [17]:
vectors_file_name = 'DV'
dv = pickle.load(open(vectors_file_name , 'rb'))

In [18]:
dv.y

['greek',
 'southern_us',
 'filipino',
 'indian',
 'indian',
 'jamaican',
 'spanish',
 'italian',
 'mexican',
 'italian',
 'italian',
 'chinese',
 'italian',
 'mexican',
 'italian',
 'indian',
 'british',
 'italian',
 'thai',
 'vietnamese',
 'thai',
 'mexican',
 'southern_us',
 'chinese',
 'italian',
 'chinese',
 'cajun_creole',
 'italian',
 'chinese',
 'mexican',
 'italian',
 'cajun_creole',
 'mexican',
 'thai',
 'italian',
 'cajun_creole',
 'italian',
 'filipino',
 'southern_us',
 'southern_us',
 'italian',
 'brazilian',
 'mexican',
 'indian',
 'mexican',
 'chinese',
 'french',
 'southern_us',
 'southern_us',
 'southern_us',
 'japanese',
 'southern_us',
 'italian',
 'southern_us',
 'italian',
 'jamaican',
 'japanese',
 'indian',
 'italian',
 'irish',
 'thai',
 'thai',
 'indian',
 'jamaican',
 'italian',
 'thai',
 'korean',
 'french',
 'french',
 'southern_us',
 'spanish',
 'indian',
 'moroccan',
 'italian',
 'italian',
 'moroccan',
 'moroccan',
 'vietnamese',
 'japanese',
 'mexican',

In [20]:
store = HDFStore('VectorSpace.h5')
store.put('d1', dv.data)
store.close()

In [25]:
dv.data.iloc[0][0].itemsize

8

In [8]:
# # Get Tf Feature Rows
# data = pd.DataFrame(dv.X)
# data.shape
# # Feature Selection
# # Drop Features with Df < 3
# data.drop([
#     col for col, val in pd.DataFrame(dv.X).sum().iteritems() if int(val) <= 3
# ],axis=1,inplace=True)
# data.shape

In [9]:
# # Tf-Idf Calculations
# data.mul(data.sum().apply(lambda df: np.log10(data.shape[0] / (df + 1))),
#          axis=1)

## Train Test Split:

In [26]:

# data = dv.data.copy()
# data = pd.DataFrame(X_self.toarray())
data = pd.DataFrame(dv.data.toarray())
data['label'] = dv.y
shuffled_data = data

shuffled_data.sample(frac=1)
train_size = 0.8
test_size = 0.2

train_data, test_data = shuffled_data.sample(frac=train_size), shuffled_data.sample(frac=test_size)
X_train, y_train = train_data.loc[:, train_data.columns != 'label'], train_data['label']
X_test, y_test = test_data.loc[:, test_data.columns != 'label'], test_data['label']

,0,1,2,3,4,5,6,7,8,9,...,3666,3667,3668,3669,3670,3671,3672,3673,3674,label
23422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,brazilian
9901,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,japanese
9693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,italian
13848,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,chinese
36791,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,chinese
15011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,spanish
2876,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,indian
31191,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,italian
29801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,chinese
11619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,cajun_creole


## Distance Formula:

In [10]:
def euclidian_distance(p1, p2):
    return np.linalg.norm(np.array(p2) - np.array(p1))


def cosine_similarity(p1, p2):
    return ((np.dot(p1, p2)) / (np.linalg.norm(p1) * np.linalg.norm(p2)))

In [11]:
def accuracy(y_test, pred):
    return len([1 for p, y in zip(pred, y_test) if p == y]) / len(pred) * 100

## k-nearest neighbors (KNN):

In [13]:
class KNNClassifier():
    def __init__(self, neighbors=3, distance_formula=euclidian_distance):
        self.distance_formula = distance_formula
        self.neighbors = neighbors
        self.X_train = []
        self.y_train = []

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X_test, k = 3):
        self.neighbors = k
        pred = []
        for index, test_row in X_test.iterrows():
            print(index)
            clear_output(wait=True)
            if self.distance_formula == euclidian_distance:
                pred.append(
                    self.X_train.apply(
                        (lambda row: self.distance_formula(row, test_row)),
                        axis=1).sort_values(ascending=True))
            else:
                pred.append(
                    self.X_train.apply(
                        (lambda row: self.distance_formula(row, test_row)),
                        axis=1).sort_values(ascending=False))

        new_pred = [x[:self.neighbors] for x in pred]
        label_pred = []
        for indexes in new_pred:
            labels = []
            #     print(indexes)
            for index, value in indexes.items():
                #         print(index)
                #         print(y[index])
                labels.append(self.y_train[index])
            label_pred.append(Counter(labels).most_common(1)[0][0])
        return pd.Series(label_pred)

In [75]:
knn = KNNClassifier(distance_formula=cosine_similarity)
knn.fit(X_train, y_train)
pred = knn.predict(X_test[:1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


In [77]:
accuracy(y_test, pred)

100.0

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
clf = RandomForestClassifier(random_state=0)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print("Random Forest")
print(accuracy_score(y_test,pred))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

Random Forest
0.928095537397863


In [23]:
get_size(clf)

12477

In [91]:
def get_query_vector(self, query_terms):
        ps = PorterStemmer()
        pd_data = pd.DataFrame(self.data.toarray())
        query_vector =pd.Series(pd_data.T[0])
        print(query_vector.index)
        query_terms = [ps.stem(word.lower()) for word in query_terms]
        for term in query_terms:
            if term in self.vocab_index.keys():
                if self.vocab_index[term] in self.idf.index:
                    print(term)
                    query_vector.loc[ self.idf.index.get_loc(self.vocab_index[term])] += 1
#         print(query_vector.col[query_vector > 0])
        for index in query_vector.index[query_vector > 0]:
            query_vector.iloc[index] *= self.idf.iloc[index] 
        return query_vector
dv.get_query_vector = get_query_vector

In [96]:
dv.vocab_index['romaine lettuc']
# dv.idf.index.get_loc()

5113

In [100]:
qury = dv.get_query_vector(dv,[ "green chile",
        "jalapeno chilies",
        "onions",
        "ground black pepper",
        "salt",
        "chopped cilantro fresh",
        "green bell pepper",
        "garlic",
        "white sugar",
        "roma tomatoes",
        "celery",
        "dried oregano"])
qury.index[qury > 0]
clf.predict([qury])[0]

RangeIndex(start=0, stop=3675, step=1)
green chil
jalapeno chili
onion
ground black pepp
salt
chopped cilantro fresh
green bell pepp
garlic
white sugar
roma tomato
celeri
dried oregano


Int64Index([ 276,  535,  705, 1088, 1219, 1438, 1441, 1522, 1554, 1559, 1594,
            1793, 2287, 2420, 2632, 2812, 2813, 2872, 2920, 3580],
           dtype='int64')

'mexican'

In [13]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train,y_train)
pred= gnb.predict(X_test)
print("GNB")
print(accuracy_score(y_test,pred))

GaussianNB(priors=None, var_smoothing=1e-09)

GNB
0.4594594594594595


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

LR = LogisticRegression()
# KNN = KNeighborsClassifier(n_neighbors=10)
svm = SVC()

LR.fit(X_train,y_train)
pred = LR.predict(X_test)

print("\nLogistic Regression")
print(accuracy_score(y_test,pred))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)


Logistic Regression
0.8500314267756128


In [ ]:
# KNN.fit(X_train,y_train)
# pred = KNN.predict(X_test)

# print("\nK neighbors")
# print(accuracy_score(y_test,pred))

svm.fit(X_train,y_train)
pred = svm.predict(X_test)
print("Support Vector Machine")
print(accuracy_score(y_test,pred))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [81]:
y_actu = pd.Categorical(y_test,categories= ['athletics', 'rugby', 'football', 'tennis', 'cricket'] )
y_pred = pd.Categorical(pred, categories= ['athletics', 'rugby', 'football', 'tennis', 'cricket'])

df_confusion = pd.crosstab(y_actu, y_pred, margins=True, rownames=['actual'], colnames=['predicted'])
df_confusion

ValueError: arrays must all be same length

In [17]:
knn = KNNClassifier(distance_formula=euclidian_distance)
knn.fit(X_train, y_train)
pred = knn.predict(X_test)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-b17341535e05>", line 3, in <module>
    pred = knn.predict(X_test)
  File "<ipython-input-13-f3c9d84dbbe0>", line 22, in predict
    axis=1).sort_values(ascending=True))
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py", line 6487, in apply
    return op.get_result()
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\apply.py", line 151, in get_result
    return self.apply_standard()
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\apply.py", line 257, in apply_standard
    self.apply_series_generator()
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\apply.py", line 286, in apply_series_generator
    results[i] = self.f(v)
  File "<ipython-input-13-f3c9d84dbbe0>", line 21, in <lambda>
    (lambda row: s

KeyboardInterrupt: 

In [ ]:
accuracy(y_test, pred)

## K-Means Clustering:

In [18]:
# PreProcessing For Kmeans
clustering_data = data
clustering_data['file_names'] = dv.doc_index.values()
clustering_data = clustering_data.sample(frac = 1)
clustering_data_labels, clustering_data_file_names = clustering_data['label'], clustering_data['file_names']
clustering_data = clustering_data.drop(['label', 'file_names'], axis = 1)
clustering_data_input = clustering_data

In [19]:
clustering_data

,0,1,2,4,5,13,16,22,23,24,...,6656,6661,6672,6673,6674,6680,6681,6682,6683,6684
36331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
16272,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17652,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3414,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20328,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
from collections import Counter


class KMeansCluster():
    def __init__(self, n_clusters=2, distance_formula=euclidian_distance):
        self.num_clusters = n_clusters
        self.n_clusters = n_clusters
        self.max_iterations = 100
        self.distance_formula = distance_formula

    def get_labels(self, X_train, centroids):
        a1 = pd.DataFrame(centroids).apply(lambda center: X_train.apply(
            (lambda row: self.distance_formula(row, center)), axis=1),
                                           axis=1)
        a2 = []
        print(a1.shape)
        sort_ascending = self.distance_formula == euclidian_distance
        for col in a1:
            if sort_ascending:
#                 print(a1[col].sort_values(ascending=True).index[:5])
                a2.append(a1[col].sort_values(ascending=True).index[0])
            else:
                a2.append(a1[col].sort_values(ascending=False).index[0])
        return a2


#     def nearest_centeroid(self, x, centroids):
#             return pd.DataFrame(centroids).apply(lambda center: euclidian_distance(x, center), axis=1).sort_values(ascending=True).index[0]

    def fit(self, X_train):
        self.X_train = X_train

        self.centroids = []
        for c in range(0, self.num_clusters):
            self.centroids.append(
                pd.DataFrame(X_train.sample(self.n_clusters).values).apply(
                    sum, axis=0) / self.n_clusters)

        def new_centroids(X, labels):

            centroids = []
            for c in range(0, self.num_clusters):
                clustered_rows = X.iloc[[
                    x[0] for x in enumerate(labels) if x[1] == c
                ]]
                if len(clustered_rows.index) == 0:
                    print("NULL")
                    centroids.append(
                        pd.DataFrame(X.sample(self.n_clusters).values).apply(
                            sum, axis=0) / self.n_clusters)
                else:
                    centroids.append(
                        clustered_rows.apply(sum) / len(clustered_rows.index))
                    centroids[c].index = range(0, len(X.columns))
            return centroids

        iterations = 0
        old_centroids = self.centroids
        while iterations < self.max_iterations:
            clear_output(wait=True)
            print(iterations)
            old_centroids = self.centroids
            labels = self.get_labels(self.X_train, self.centroids)
            self.centroids = new_centroids(self.X_train, labels)

            converged = True
            for c in range(0, len(self.centroids)):
                if not self.centroids[c].equals(old_centroids[c]):
                    converged = False
                    break
            if converged:
                break

            iterations += 1
        self.labels = labels
        self.cluster_centers_ = self.centroids

    def predict(self, X_test):
        #         return  X_test.apply(lambda x : self.nearest_centeroid(x, self.centroids), axis= 1)
        return np.array(self.get_labels(X_test, self.centroids))

    def purity(self, labels):
        total = 0
        for c in range(0, self.n_clusters):
            la = labels.iloc[[
                x[0] for x in enumerate(self.labels) if x[1] == c
            ]]
            if len(la) != 0:
                print(f'Cluster {c} size : {len(la)}')
                print(f'Cluster {c} Most Common Label : {Counter(la).most_common(1)[0][0]}')
                print(f'Cluster {c} Most Common Label Count : {Counter(la).most_common(1)[0][1]}')
                print(Counter(la).most_common(1)[0][1] / len(la))
                total += Counter(la).most_common(1)[0][1]

        purity = total / self.X_train.shape[0]
        print()
        print(f'Purity : {purity}')
        return purity

In [21]:
%%time
kmeans = KMeansCluster(n_clusters=5, distance_formula=cosine_similarity)
kmeans.fit(clustering_data)

0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\apply.py", line 251, in apply_standard
    labels=labels)
  File "pandas/_libs/reduction.pyx", line 641, in pandas._libs.reduction.reduce
  File "pandas/_libs/reduction.pyx", line 146, in pandas._libs.reduction.Reducer.get_result
  File "<ipython-input-20-70992e34be4c>", line 13, in <lambda>
    (lambda row: self.distance_formula(row, center)), axis=1),
  File "<ipython-input-11-c22a7b6cc14f>", line 6, in cosine_similarity
    return ((np.dot(p1, p2)) / (np.linalg.norm(p1) * np.linalg.norm(p2)))
  File "C:\ProgramData\Anaconda3\lib\site-packages\numpy\linalg\linalg.py", line 2450, in norm
    sqnorm = dot(x, x)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\magics\execution.py", line 1315, in time
    out = eval(code_2, glob, local_ns)
  File "<timed e

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\apply.py", line 251, in apply_standard
    labels=labels)
  File "pandas/_libs/reduction.pyx", line 641, in pandas._libs.reduction.reduce
  File "pandas/_libs/reduction.pyx", line 146, in pandas._libs.reduction.Reducer.get_result
  File "<ipython-input-20-70992e34be4c>", line 13, in <lambda>
    (lambda row: self.distance_formula(row, center)), axis=1),
  File "<ipython-input-11-c22a7b6cc14f>", line 6, in cosine_similarity
    return ((np.dot(p1, p2)) / (np.linalg.norm(p1) * np.linalg.norm(p2)))
  File "C:\ProgramData\Anaconda3\lib\site-packages\numpy\linalg\linalg.py", line 2450, in norm
    sqnorm = dot(x, x)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\magics\execution.py", line 1315, in time
    out = eval(code_2, glob, local_ns)
  File "<timed e

TypeError: can only concatenate str (not "list") to str

In [ ]:
kmeans.purity(clustering_data_labels)

In [209]:
# Documents in cluster
for c in range(0, kmeans.n_clusters):
    print(f'Documents in Cluster {c}')
    print(clustering_data_file_names.iloc[[x[0] for x in enumerate(kmeans.labels) if x[1] == c ]])

Documents in Cluster 0
404     football\180.txt
419     football\195.txt
571        rugby\082.txt
245     football\021.txt
99     athletics\100.txt
300     football\076.txt
440     football\216.txt
595        rugby\106.txt
421     football\197.txt
613        rugby\124.txt
258     football\034.txt
591        rugby\102.txt
535        rugby\046.txt
227     football\003.txt
350     football\126.txt
268     football\044.txt
603        rugby\114.txt
441     football\217.txt
392     football\168.txt
425     football\201.txt
462     football\238.txt
480     football\256.txt
405     football\181.txt
331     football\107.txt
452     football\228.txt
228     football\004.txt
680       tennis\044.txt
294     football\070.txt
325     football\101.txt
575        rugby\086.txt
             ...        
384     football\160.txt
615        rugby\126.txt
273     football\049.txt
342     football\118.txt
387     football\163.txt
395     football\171.txt
329     football\105.txt
451     football\227.txt
40

## Saving Trained Models:

In [168]:
# Note : KNN has no trainng step
knn_file_name = 'KNN.sav'
kmeans_file_name = 'KMeans.sav'
joblib.dump(knn, knn_file_name)
joblib.dump(kmeans, kmeans_file_name)

['KNN.sav']

['KMeans.sav']